# Bench

## Base sim (random input)

In [2]:
from pokerl.env.rewards.basic_rewards import PokemonBlueEnv
import random

import ray

In [3]:
ray.init(runtime_env={"working_dir": "../"})

2024-01-15 20:39:06,523	ERROR services.py:1329 -- Failed to start the dashboard , return code 1
2024-01-15 20:39:06,523	ERROR services.py:1354 -- Error should be written to 'dashboard.log' or 'dashboard.err'. We are printing the last 20 lines for you. See 'https://docs.ray.io/en/master/ray-observability/ray-logging.html#logging-directory-structure' to find where the log file is.
2024-01-15 20:39:06,532	ERROR services.py:1398 -- 
The last 20 lines of C:\Users\kaosl\AppData\Local\Temp\ray\session_2024-01-15_20-39-04_280478_3512\logs\dashboard.log (it contains the error message from the dashboard): 
  File "c:\Users\kaosl\AppData\Local\pypoetry\Cache\virtualenvs\pokerl-qa45OdQw-py3.10\lib\site-packages\ray\dashboard\head.py", line 325, in run
    modules = self._load_modules(self._modules_to_load)
  File "c:\Users\kaosl\AppData\Local\pypoetry\Cache\virtualenvs\pokerl-qa45OdQw-py3.10\lib\site-packages\ray\dashboard\head.py", line 219, in _load_modules
    head_cls_list = dashboard_utils.ge

: 

In [ ]:
# Hyperparameters
max_timesteps = 1000

In [12]:
def simulate():
    env = PokemonBlueGym()
    for _ in (range(max_timesteps)):
        action = random.choice(env.action_space_convertissor)
        env.step(action)
    return env.render()

@ray.remote
def simulate_ray():
    simulate()

In [18]:
%%time
for i in range(20):
    simulate()

CPU times: total: 1min 23s
Wall time: 1min 49s


In [20]:
%%time
results = ray.get([simulate_ray.remote() for _ in range(40)])

CPU times: total: 78.1 ms
Wall time: 45.2 s


## Bench successive append on numpy array vs list

In [24]:
import random

amount_of_data = 100

state = [[[random.randint(0, 255) for _ in range(100)] for _ in range(100)] for _ in range(amount_of_data)]
action = [random.randint(0, 10) for _ in range(amount_of_data)]
reward = [random.randint(0, 1) for _ in range(amount_of_data)]
next_state = [[[random.randint(0, 255) for _ in range(100)] for _ in range(100)] for _ in range(amount_of_data)]
done = [0 for _ in range(amount_of_data)]

In [25]:
%%time
from collections import deque

memory = deque(maxlen=amount_of_data*1000)
for i in range(amount_of_data*1000):
    i_ = i % amount_of_data
    memory.append((state[i_], action[i_], reward[i_], next_state[i_], done[i_]))


CPU times: total: 31.2 ms
Wall time: 48.5 ms


In [26]:
%%time
import numpy as np
memory = np.zeros((amount_of_data*1000, 5, 100, 100))
for i in range(amount_of_data):
    memory[i, 0] = state[i]
    memory[i, 1] = action[i]
    memory[i, 2] = reward[i]
    memory[i, 3] = next_state[i]
    memory[i, 4] = done[i]


CPU times: total: 62.5 ms
Wall time: 467 ms


Conclusion: deque is fastest